In [63]:
import os
import re
import time
import imageio
import json
import glob
from pathlib import Path
from datetime import datetime, timedelta

import geojsoncontour
import numpy as np
import pandas as pd

import wrf
from netCDF4 import Dataset as NetCDFFile

import folium
from folium import plugins
import plotly.graph_objects as go

import matplotlib.pyplot as plt
import branca.colormap as cm
from matplotlib import colors as mcolors

from shapely.geometry import shape, Point

In [64]:
nc_file = NetCDFFile('../wrf_output')
time_size = nc_file.dimensions['Time'].size

In [3]:
with open('../Limite Distrito de Barranquilla.geojson') as f:
    baq_geojson = json.load(f)
    baq_polygon = shape(baq_geojson['features'][0]['geometry'])

In [65]:
# https://wrf-python.readthedocs.io/en/latest/user_api/generated/wrf.wrf.getvar.html
height = wrf.getvar(nc_file, 'height', timeidx=1)
Var_all = wrf.getvar(nc_file, 'tc', timeidx=1)

var = wrf.interplevel(Var_all, height, 150)

(lats, lons) = wrf.latlon_coords(var)

var

<xarray.DataArray 'temp_interp' (south_north: 81, west_east: 90)>
array([[      nan,       nan,       nan, ..., 25.10964 , 25.10948 ,
        25.109318],
       [      nan,       nan,       nan, ..., 25.114904, 25.114868,
        25.114006],
       [      nan,       nan,       nan, ..., 25.121363, 25.1193  ,
        25.118526],
       ...,
       [25.236725, 25.234795, 25.231813, ..., 25.060196, 25.060104,
        25.057823],
       [25.233667, 25.233133, 25.231472, ..., 25.056171, 25.05298 ,
        25.050219],
       [25.231842, 25.2309  , 25.229963, ..., 25.047691, 25.045143,
        25.042477]], dtype=float32)
Coordinates:
    XLONG    (south_north, west_east) float32 -74.96 -74.96 ... -74.68 -74.68
    XLAT     (south_north, west_east) float32 10.85 10.85 10.85 ... 11.11 11.11
    XTIME    float32 180.0
    Time     datetime64[ns] 2022-04-04T12:00:00
    level    int64 150
Dimensions without coordinates: south_north, west_east
Attributes:
    FieldType:      104
    units:          degC
    stagger:        
    coordinates:    XLONG XLAT XTIME
    projection:     LatLon(stand_lon=-74.8219985961914, moad_cen_lat=10.98099...
    _FillValue:     9.969209968386869e+36
    missing_value:  9.969209968386869e+36
    vert_units:     m

In [68]:
var_geo_bounds = wrf.geo_bounds(var)

In [70]:
len(np.arange(var_geo_bounds.bottom_left.lat, var_geo_bounds.top_right.lat, 0.01))

26

In [71]:
len(np.arange(var_geo_bounds.bottom_left.lon, var_geo_bounds.top_right.lon, 0.02))

15

In [72]:
np.arange(var_geo_bounds.bottom_left.lat, var_geo_bounds.top_right.lat, 0.01)

array([10.85280037, 10.86280037, 10.87280037, 10.88280037, 10.89280037,
       10.90280037, 10.91280037, 10.92280037, 10.93280037, 10.94280037,
       10.95280037, 10.96280037, 10.97280037, 10.98280037, 10.99280037,
       11.00280037, 11.01280037, 11.02280037, 11.03280037, 11.04280037,
       11.05280037, 11.06280037, 11.07280037, 11.08280037, 11.09280037,
       11.10280037])

In [73]:
np.arange(var_geo_bounds.bottom_left.lon, var_geo_bounds.top_right.lon, 0.02)

array([-74.96463013, -74.94463013, -74.92463013, -74.90463013,
       -74.88463013, -74.86463013, -74.84463013, -74.82463013,
       -74.80463013, -74.78463013, -74.76463013, -74.74463013,
       -74.72463013, -74.70463013, -74.68463013])

In [60]:
def get_data(nc_file: NetCDFFile, timeidx: int):
    dx = 150
    height = wrf.getvar(nc_file, 'height', timeidx=timeidx)

    u_all = wrf.getvar(nc_file, 'ua', timeidx=timeidx)
    v_all = wrf.getvar(nc_file, 'va', timeidx=timeidx)
    T_all = wrf.getvar(nc_file, 'tc', timeidx=timeidx)
    P_all = wrf.getvar(nc_file, 'pressure', timeidx=timeidx)
    pw = wrf.getvar(nc_file, 'pw', timeidx=timeidx)

    P = wrf.interplevel(P_all, height, dx)
    T = wrf.interplevel(T_all, height, dx)
    u = wrf.interplevel(u_all, height, dx)
    v = wrf.interplevel(v_all, height, dx)

    data = {
        'pwater': ('Precipitable Water (kg/m2)', pw),
        'temp': ('Temperature (C)', T),
        'wind': ('Wind speed (m/s)', np.sqrt(u ** 2 + v ** 2)),
        'uwind': ('U wind speed (m/s)', u),
        'vwind': ('V wind speed (m/s)', v),
        'press': ('Pressure (hPa)', P)
    }

    return data


def geojson_title_to_float(title):
    result = re.search(
        r"([-]?([0-9]*[.])?[0-9]+)-([-]?([0-9]*[.])?[0-9]+)", title)
    groups = result.groups()

    value = np.median([float(groups[0]), float(groups[2])])

    return value


def gj_to_df(gj):
    gj_data = np.zeros([len(gj['features']), 2])

    for i in range(len(gj['features'])):
        gj['features'][i]['id'] = i
        gj_data[i, 0] = i
        gj_data[i, 1] = geojson_title_to_float(
            gj['features'][i]['properties']['title'])

    df = pd.DataFrame(gj_data, columns=['id', 'variable'])

    return df


def build_gif_frame(lats, lons, caption, variable, date):
    contour = plt.contourf(lons, lats, variable, cmap=plt.cm.jet)

    gj = json.loads(geojsoncontour.contourf_to_geojson(
        contourf=contour, ndigits=4, unit='m'))
    df_contour = gj_to_df(gj)
    
    zmin = df_contour.variable.min() - df_contour.variable.median() / 10
    zmax = df_contour.variable.max() + df_contour.variable.median() / 10

    trace = go.Choroplethmapbox(
        geojson=gj,
        locations=df_contour.id,
        z=df_contour.variable,
        zmin=zmin,
        zmax=zmax,
        colorscale='jet',
        marker_line_width=0.1,
        marker=dict(opacity=0.2)
    )

    layout = go.Layout(
        title=f"{caption} - {date} GMT-5",
        title_x=0.5,
        width=600,
        margin=dict(t=26, b=0, l=0, r=0),
        font=dict(color='black', size=10),
        mapbox=dict(
            center=dict(
                lat=lats.mean().item(0),
                lon=lons.mean().item(0)
            ),
            zoom=11,
            style='carto-positron'
        )
    )

    fig = go.Figure(data=[trace], layout=layout)

    return fig


def get_image(timeidx: int, nc_var: str, start_date: datetime):
    date = start_date + timedelta(hours=timeidx * 3) - timedelta(hours=5)

    data = get_data(nc_file, timeidx)

    (caption, variable) = data[nc_var]
    (lats, lons) = wrf.latlon_coords(variable)

    fig = build_gif_frame(lats, lons, caption, variable, date)

    png_file = f"{nc_var}_{timeidx}.png"
    try:
        fig.write_image(png_file)
    except Exception:
        return None

    img = imageio.imread(png_file)
    os.remove(png_file)

    if timeidx == time_size - 1:
        build_folium_map(lats, lons, caption, variable, date)

    return img


def build_folium_map(lats, lons, caption, variable, date):
    vmin = variable.min() - variable.median() / 10
    vmax = variable.max() + variable.median() / 10

    contour = plt.contourf(lons, lats, variable, cmap=plt.cm.jet, vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(contour)

    gj = json.loads(geojsoncontour.contourf_to_geojson(
        contourf=contour, ndigits=4, unit='m'))

    f_map = folium.Map(
        location=[lats.mean(), lons.mean()],
        tiles='Cartodb Positron',
        zoom_start=12
    )

    folium.GeoJson(
        gj,
        style_function=lambda x: {
            'color': x['properties']['stroke'],
            'weight': x['properties']['stroke-width'],
            'fillColor': x['properties']['fill'],
            'opacity': 0.3
        },
        name='contour'
    ).add_to(f_map)
    
    colormap = cm.LinearColormap(
        colors=['darkblue', 'blue', 'cyan', 'green', 'greenyellow', 'yellow', 'orange', 'red', 'darkred'],
        index=np.array(cbar.values),
        vmin=cbar.values[0],
        vmax=cbar.values[len(cbar.values) - 1],
        caption=caption
    )
    f_map.add_child(colormap)
    
    folium.GeoJson(
        baq_geojson,
        style_function=lambda x: {
            'color': 'rgb(12, 131, 242)',
            'fillColor': 'rgba(255, 0, 0, 0)'
        },
        name='baq_map'
    ).add_to(f_map)
    
    var_geo_bounds = wrf.geo_bounds(variable)
    for lat in np.arange(var_geo_bounds.bottom_left.lat, var_geo_bounds.top_right.lat, 0.01):
        for lon in np.arange(var_geo_bounds.bottom_left.lon, var_geo_bounds.top_right.lon, 0.02):
            if baq_polygon.contains(Point(lon, lat)):
                x, y = wrf.ll_to_xy(nc_file, lat, lon)
                value = round(variable[x.item(0), y.item(0)].values.item(0), 2)
                folium.Marker(
                    location=[lat, lon],
                    popup=None,
                    icon=folium.DivIcon(
                    html=f"""<span style="font-size: 16px; color: yellow; -webkit-text-stroke: 1px black;">{value}</span>""")
                ).add_to(f_map)
        
    f_map.get_root().html.add_child(folium.Element('<p style="text-align:center;font-size:14px;margin:4px">{} GMT-5</p>'.format(date)))

    f_map.save(f"{nc_var}.html")

In [61]:
nc_var = 'temp'
start_date = datetime.strptime("2022-03-26 18", '%Y-%m-%d %H')

results = [get_image(timeidx, nc_var, start_date) for timeidx in range(time_size)]

imageio.mimwrite(f"{nc_var}.gif", [img for img in results if img is not None], fps=0.5)